In [1]:
from source import *

# ex = "a_example"
# ex = "b_little_bit_of_everything.in"
# ex = "c_many_ingredients.in"
# ex = "d_many_pizzas.in"
ex = "e_many_teams.in"

filename = f'input/{ex}'
load_best = True

# load data
frigo, pizza_gate = get_data(filename)

In [ ]:


# load best solution
if load_best:
    filename_best = f"outputs_best/{ex.split('.')[0]}.out"
    init_solution = load_best_to_start(filename_best)
    print('> best solution loaded !') 
else:
    init_solution = [[], [], []]

# search 
nb_voisins = 5
nb_iterations = 1000
size_tabu_list = nb_iterations
perturbation = 100
nb_iter_without_improv = 50

actual_solution = deepcopy(init_solution)
best_solution = deepcopy(init_solution)
best_score = get_score(actual_solution, frigo)
print(f'> initial score : {best_score}')

tabu_list = [actual_solution]
count_iter_no_imp = 0

for i in range(nb_iterations):
    print(f"Iter: {i}")
    list_voisins = []
    nb_try = 0
    while len(list_voisins) < nb_voisins and nb_try < 10:
        v = Voisin(actual_solution, frigo, pizza_gate, perturbation=perturbation)
        if v.solution in tabu_list:
            nb_try += 1
        else:
            list_voisins.append(v)
            # tabu_list = [v.solution] + tabu_list[:size_tabu_list-1]
            nb_try = 0 
    if not list_voisins:
        print(f"\t> empty neighborhood !")
        break
    best_v = list_voisins[np.argmax([v.score for v in list_voisins])]   
    actual_solution = deepcopy(best_v.solution)
    tabu_list = [actual_solution] + tabu_list[:size_tabu_list-1]

    if best_v.score > best_score:
        best_solution = deepcopy(actual_solution)
        best_score = best_v.score
        count_iter_no_imp=0
        print(f"\t> New best score: {best_v.score} (iter: {i})")

        # export best solution 
        export=[]
        for i,t in enumerate(best_solution):
            for d in t:
                export.append(str(i+2)+' '+' '.join(d))
        definitive_pizza_solution, shipped_piz = consolidate_results(export)
        format_resutlts(definitive_pizza_solution, ex)
        
    else:
        count_iter_no_imp+=1

    if count_iter_no_imp == nb_iter_without_improv:
        actual_solution = deepcopy(best_solution)
        count_iter_no_imp = 0
        print(f'\t> restart from best')

print(f'> final best score : {best_score}')